# Using autodiff to check gradient/Hessians

In [1]:
using Revise
using DataFrames, Random, GLM, QuasiCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices
using BenchmarkTools
using SnpArrays
using ForwardDiff
# using MendelPlots
ENV["COLUMNS"] = 240


BLAS.set_num_threads(1)
Threads.nthreads()

function simulate_random_snparray(s::Union{String, UndefInitializer}, n::Int64,
    p::Int64; mafs::Vector{Float64}=zeros(Float64, p), min_ma::Int = 5)

    #first simulate a random {0, 1, 2} matrix with each SNP drawn from Binomial(2, r[i])
    A1 = BitArray(undef, n, p) 
    A2 = BitArray(undef, n, p) 
    for j in 1:p
        minor_alleles = 0
        maf = 0
        while minor_alleles <= min_ma
            maf = 0.5rand()
            for i in 1:n
                A1[i, j] = rand(Bernoulli(maf))
                A2[i, j] = rand(Bernoulli(maf))
            end
            minor_alleles = sum(view(A1, :, j)) + sum(view(A2, :, j))
        end
        mafs[j] = maf
    end

    #fill the SnpArray with the corresponding x_tmp entry
    return _make_snparray(s, A1, A2)
end

function _make_snparray(s::Union{String, UndefInitializer}, A1::BitArray, A2::BitArray)
    n, p = size(A1)
    x = SnpArray(s, n, p)
    for i in 1:(n*p)
        c = A1[i] + A2[i]
        if c == 0
            x[i] = 0x00
        elseif c == 1
            x[i] = 0x02
        elseif c == 2
            x[i] = 0x03
        else
            throw(MissingException("matrix shouldn't have missing values!"))
        end
    end
    return x
end

┌ Info: Precompiling QuasiCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1423
[ Info: Compiling VCF parser...


_make_snparray (generic function with 1 method)

In [2]:
function simulate_VC_longitudinal(;
    n = 1000, # sample size
    d = 5, # number of observations per sample
    p = 3, # number of nongenetic covariates, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = 10, # number of causal SNPs
    seed = 2022,
    y_distribution = Bernoulli,
    T = Float64,
    )
    m == 1 || m == 2 || error("m (number of VC) must be 1 or 2")
    
    # non-genetic effect sizes
    Random.seed!(seed)
    βtrue = [1.0; rand(-0.03:0.06:0.03, p-1)]
    dist = y_distribution()
    link = canonicallink(dist)
    Dist = typeof(dist)
    Link = typeof(link)

    # variance components
    θtrue = fill(0.1, m)
    V1 = ones(d, d)
    V2 = Matrix(I, d, d)
    Γ = m == 1 ? θtrue[1] * V1 : θtrue[1] * V1 + θtrue[2] * V2

    # simulate design matrices
    Random.seed!(seed)
    X_full = [hcat(ones(d), randn(d, p - 1)) for i in 1:n]

    # simulate random SnpArray with 100 SNPs and randomly choose k SNPs to be causal
    Random.seed!(2022)
    G = simulate_random_snparray(undef, n, q)
    Gfloat = convert(Matrix{T}, G, center=true, scale=false)
    γtrue = zeros(q)
    γtrue[1:k] .= rand([-0.2, 0.2], k)
    shuffle!(γtrue)
    η_G = Gfloat * γtrue

    # simulate phenotypes
    if y_distribution == Normal
        τtrue = 10.0
        σ2 = inv(τtrue)
        σ = sqrt(σ2)
        obs = Vector{GaussianCopulaVCObs{T}}(undef, n)
        for i in 1:n
            X = X_full[i]
            η = X * βtrue
            η .+= η_G[i] # add genetic effects
            μ = GLM.linkinv.(link, η)
            vecd = Vector{ContinuousUnivariateDistribution}(undef, d)
            for i in 1:d
                vecd[i] = y_distribution(μ[i], σ)
            end
            nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
            # simuate single vector y
            y = Vector{T}(undef, d)
            res = Vector{T}(undef, d)
            rand(nonmixed_multivariate_dist, y, res)
            V = m == 1 ? [V1] : [V1, V2]
            obs[i] = GaussianCopulaVCObs(y, X, V)
        end
        qc_model = GaussianCopulaVCModel(obs)
    else
        obs = Vector{GLMCopulaVCObs{T, Dist, Link}}(undef, n)
        for i in 1:n
            X = X_full[i]
            η = X * βtrue
            η .+= η_G[i] # add genetic effects
            μ = GLM.linkinv.(link, η)
            vecd = Vector{DiscreteUnivariateDistribution}(undef, d)
            for i in 1:d
                vecd[i] = y_distribution(μ[i])
            end
            nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
            # simuate single vector y
            y = Vector{T}(undef, d)
            res = Vector{T}(undef, d)
            rand(nonmixed_multivariate_dist, y, res)
            V = m == 1 ? [V1] : [V1, V2]
            obs[i] = GLMCopulaVCObs(y, X, V, dist, link)
        end
        qc_model = GLMCopulaVCModel(obs)
    end
    return qc_model, Γ, G, βtrue, θtrue, γtrue
end

k = 0 # number of causal SNPs

qc_model, Γ, G, βtrue, θtrue, γtrue = simulate_VC_longitudinal(
    n = 5000, # sample size
    d = 5, # number of observations per sample
    p = 3, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 1000,
    y_distribution = Bernoulli,
    T = Float64,
)

@show qc_model;

qc_model = Quasi-Copula Variance Component Model
  * base distribution: Bernoulli
  * link function: LogitLink
  * number of clusters: 5000
  * cluster size min, max: 5, 5
  * number of variance components: 2
  * number of fixed effects: 3



In [3]:
@time optm = QuasiCopula.fit!(qc_model,
    Ipopt.IpoptSolver(
        print_level = 5, 
        tol = 10^-6, 
        max_iter = 1000,
        accept_after_max_steps = 4,
        warm_start_init_point="yes", 
        limited_memory_max_history = 6, # default value
        hessian_approximation = "limited-memory",
#         derivative_test="second-order"
    )
);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        5
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [4]:
@show βtrue
@show qc_model.β
@show qc_model.∇β

@show θtrue
@show qc_model.θ
@show qc_model.∇θ;

βtrue = [1.0, 0.03, 0.03]
qc_model.β = [1.0114777784855657, 0.02482165570715728, 0.006462230525953184]
qc_model.∇β = [-1.224434083013648e-7, 6.693747525154947e-7, -3.092028739212077e-7]
θtrue = [0.1, 0.1]
qc_model.θ = [0.0973239094129173, 0.12418281619873915]
qc_model.∇θ = [3.02541980801152e-7, 9.500306119569757e-8]


## Is $\nabla_\beta res$ calculated correctly? 

We can check using ForwardDiff

The function is 

$$res_{ij}(\beta) = \frac{y_{ij} - \mu_{ij}}{\sqrt{\sigma_{ij}^2(\beta)}}$$

### Normal

Assumes y, X are given. We calculate the residuals for just 1 sample

In [2]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

# objective
function resβ(β)
    η = X * β # d by 1
    μ = GLM.linkinv.(IdentityLink(), η)
    varμ = GLM.glmvar.(Normal(), μ)
    return (y - μ) ./ sqrt.(varμ) # d by 1
end

# mathematical gradient
function ∇resβ(β)
    d, p = size(X)
    ∇resβ = zeros(d, p)
    for i in 1:p, j in 1:d
        ∇resβ[j, i] = -X[j, i]
    end
    return ∇resβ # d × p
end

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)

# random beta vector
β = rand(size(qc_model.data[1].X, 2))

# check objective
@show resβ(β)

# compare mathematical and numerical gradient
[vec(∇resβ(β)) vec(∇resβ_autodiff(β))]

resβ(β) = [-1.5263015405222384, -2.6945001310537258, -1.9847678519577736, -0.900074590336336]


12×2 Matrix{Float64}:
 -1.0        -1.0
 -1.0        -1.0
 -1.0        -1.0
 -1.0        -1.0
  2.07458     2.07458
 -1.94686    -1.94686
  0.0808759   0.0808759
  0.154606    0.154606
 -0.931964   -0.931964
 -2.26098    -2.26098
 -1.19819    -1.19819
  0.0763038   0.0763038

### Bernoulli

In [8]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

# objective
function resβ(β)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogitLink(), η)
    varμ = GLM.glmvar.(Bernoulli(), μ)
    return (y - μ) ./ sqrt.(varμ) # d by 1
end

# mathematical gradient
function ∇resβ(β::AbstractVector{T}) where T
    d, p = size(X)
    ∇resβ = zeros(T, d, p)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogitLink(), η) # d by 1
    varμ = GLM.glmvar.(Bernoulli(), μ) # d by 1
    res = (y - μ) ./ sqrt.(varμ) # d by 1
    for i in 1:p, j in 1:d
        varμ_j = varμ[j]
        x_ji = X[j, i]
        res_j = res[j]
        μ_j = μ[j]
        ∇resβ[j, i] = -sqrt(varμ_j) * x_ji - 
            (0.5 * res_j * (1 - 2μ_j) * x_ji)
    end
    return ∇resβ # d × p
end
∇²resβ_autodiff = x -> ForwardDiff.jacobian(∇resβ, x)

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)

# check objective
@show resβ(qc_model.β)

# compare mathematical and numerical gradient
[vec(∇resβ(qc_model.β)) vec(∇resβ_autodiff(qc_model.β))]


resβ(qc_model.β) = [0.614284473130316, 0.5863886158923909, 0.60381428358954, 0.6033000519453668, 0.5941433563556224]


15×2 Matrix{Float64}:
 -0.307142   -0.307142
 -0.293194   -0.293194
 -0.301907   -0.301907
 -0.30165    -0.30165
 -0.297072   -0.297072
  0.63719     0.63719
 -0.570809   -0.570809
  0.024417    0.024417
  0.046637    0.046637
 -0.27686    -0.27686
 -0.694442   -0.694442
 -0.351303   -0.351303
  0.0230367   0.0230367
 -0.141952   -0.141952
 -0.3061     -0.3061

### Poisson

In [61]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

# objective
function resβ(β)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogLink(), η)
    varμ = GLM.glmvar.(Poisson(), μ)
    return (y - μ) ./ sqrt.(varμ) # d by 1
end

# mathematical gradient
function ∇resβ(β)
    d, p = size(X)
    ∇resβ = zeros(d, p)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogLink(), η) # d by 1
    varμ = GLM.glmvar.(Poisson(), μ) # d by 1
    res = (y - μ) ./ sqrt.(varμ) # d by 1
    dμ = GLM.mueta.(LogLink(), η) # d by 1
    for i in 1:p, j in 1:d
        varμ_j = varμ[j]
        x_ji = X[j, i]
        res_j = res[j]
        μ_j = μ[j]
        dμ_j = dμ[j]
        ∇resβ[j, i] = x_ji * (-(inv(sqrt(varμ_j)) + (0.5 * inv(varμ_j)) * res_j) * dμ_j)
    end
    return ∇resβ # d × p
end

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)

# random beta vector
β = rand(size(qc_model.data[1].X, 2))

# check objective
@show resβ(β)

# compare mathematical and numerical gradient
[vec(∇resβ(β)) vec(∇resβ_autodiff(β))]

resβ(β) = [0.8012638765796852, -6.734952547066679, -1.3994071698413866, -0.48023808695797]


12×2 Matrix{Float64}:
 -1.07727    -1.07727
 -3.65238    -3.65238
 -1.22049    -1.22049
 -1.02842    -1.02842
  2.23488     2.23488
 -7.11068    -7.11068
  0.0987079   0.0987079
  0.159001    0.159001
 -1.00397    -1.00397
 -8.25796    -8.25796
 -1.46237    -1.46237
  0.0784727   0.0784727

## Check $\nabla_\beta L$

In [38]:
function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

function loglikelihood(
    β::AbstractVector{T}, 
    qc_model::Union{GLMCopulaVCModel, NBCopulaVCModel}
    ) where T
    θ = qc_model.θ
    # allocate vector of type T
    n, p = size(qc_model.data[1].X)
    η = zeros(T, n)
    μ = zeros(T, n)
    varμ = zeros(T, n)
    res = zeros(T, n)
    storage_n = zeros(T, n)
    q = zeros(T, length(θ))
    logl = zero(T)
    for gc in qc_model.data
        X = gc.X
        y = gc.y
        n, p = size(X)
        # update_res! step (need to avoid BLAS)
        A_mul_b!(η, X, β)
        for i in 1:gc.n
            μ[i] = GLM.linkinv(gc.link, η[i])
            varμ[i] = GLM.glmvar(gc.d, μ[i]) # Note: for negative binomial, d.r is used
#             dμ[i] = GLM.mueta(gc.link, η[i])
#             w1[i] = dμ[i] / varμ[i]
#             w2[i] = w1[i] * dμ[i]
            res[i] = y[i] - μ[i]
        end
        # standardize_res! step
        for j in eachindex(y)
            res[j] /= sqrt(varμ[j])
        end
        # std_res_differential! step (this will compute ∇resβ)
#         for i in 1:gc.p
#             for j in 1:gc.n
#                 ∇resβ[j, i] = -sqrt(varμ[j]) * X[j, i] - (0.5 * res[j] * (1 - (2 * μ[j])) * X[j, i])
#             end
#         end
        # update Γ
        @inbounds for k in 1:gc.m
            A_mul_b!(storage_n, gc.V[k], res)
            q[k] = dot(res, storage_n) / 2 # q[k] = 0.5 r' * V[k] * r (update variable b for variance component model)
        end
        # component_loglikelihood
        for j in 1:gc.n
            logl += QuasiCopula.loglik_obs(gc.d, y[j], μ[j], one(T), one(T))
        end
        tsum = dot(θ, gc.t)
        logl += -log(1 + tsum)
        qsum  = dot(θ, q) # qsum = 0.5 r'Γr
        logl += log(1 + qsum)
    end
    return logl
end

function loglikelihood(
    β::AbstractVector{T}, 
    gcm::GaussianCopulaVCModel
    ) where T
    θ = gcm.θ
    τ = gcm.τ[1]
    # allocate vector of type T
    n, p = size(gcm.data[1].X)
    μ = zeros(T, n)
    res = zeros(T, n)
    storage_n = zeros(T, n)
    q = zeros(T, length(θ))
    logl = zero(T)
    for gc in gcm.data
        X = gc.X
        y = gc.y
        n, p = size(X)
        sqrtτ = sqrt(abs(τ))
        # update_res! step (need to avoid BLAS)
        A_mul_b!(μ, X, β)
        for i in 1:gc.n
            res[i] = y[i] - μ[i]
        end
        # standardize_res! step
        res .*= sqrtτ
        rss  = abs2(norm(res)) # RSS of standardized residual
        tsum = dot(abs.(θ), gc.t) # ben: why is there abs here?
        logl += - log(1 + tsum) - (gc.n * log(2π) -  gc.n * log(abs(τ)) + rss) / 2
        # update Γ
        @inbounds for k in 1:gc.m
            A_mul_b!(storage_n, gc.V[k], res)
            q[k] = dot(res, storage_n) / 2 # q[k] = 0.5 r' * V[k] * r (update variable b for variance component model)
        end
        qsum  = dot(θ, q)
        logl += log(1 + qsum)
    end
    return logl
end

# sample data
autodiff_loglikelihood(β) = loglikelihood(β, qc_model)

autodiff_loglikelihood (generic function with 1 method)

In [30]:
# autodiff Gradient
∇logl = x -> ForwardDiff.gradient(autodiff_loglikelihood, x)
∇βtrue = ∇logl(qc_model.β)

3-element Vector{Float64}:
 -1.224433049396012e-7
  6.693747398034411e-7
 -3.0920281643420644e-7

In [31]:
# gradient from math
loglikelihood!(qc_model, true, false)
∇βobs = qc_model.∇β

3-element Vector{Float64}:
 -1.224434083013648e-7
  6.693747525154947e-7
 -3.092028739212077e-7

## Check $\nabla_\beta^2 L$

Hessians for a single observation seems to differ quite a bit

In [28]:
function two_term_Hessian(gcm::Union{GLMCopulaVCModel, NBCopulaVCModel})
    p = length(gcm.β)
    T = eltype(gcm.β)
    H = zeros(T, p, p)
    for gc in gcm.data
        d = gc.n # number of observations for current sample
        # GLM term
        H -= Transpose(gc.X) * Diagonal(gc.w2) * gc.X
        # trailing terms
        res = gc.res # d × 1 standardized residuals
        ∇resβ = gc.∇resβ # d × p
        Γ = zeros(T, d, d)
        for k in 1:gc.m # loop over variance components
            Γ .+= gcm.θ[k] .* gc.V[k]
        end
        denom = abs2(1 + 0.5 * (res' * Γ * res))
        H -= (∇resβ' * Γ * res) * (∇resβ' * Γ * res)' / denom
    end
    return H
end

function two_term_Hessian(gcm::GaussianCopulaVCModel)
    p = length(gcm.β)
    T = eltype(gcm.β)
    H = zeros(T, p, p)
    for gc in gcm.data
        d = gc.n # number of observations for current sample
        # GLM term
        H -= Transpose(gc.X) * gc.X
        # trailing terms
        res = gc.res # d × 1 standardized residuals
        ∇resβ = -sqrt(gcm.τ[1]) .* gc.X # d × p
        Γ = zeros(T, d, d)
        for k in 1:gc.m # loop over variance components
            Γ .+= gcm.θ[k] .* gc.V[k]
        end
        denom = abs2(1 + 0.5 * (res' * Γ * res))
        H -= (∇resβ' * Γ * res) * (∇resβ' * Γ * res)' / denom
    end
    return H
end

function three_term_hessian(qc_model::Union{GLMCopulaVCModel, NBCopulaVCModel})
#     # sarah's implementation
#     loglikelihood!(qc_model, true, true)
#     return qc_model.Hβ
#     @show qc_model.Hβ
    p = length(qc_model.β)
    T = eltype(qc_model.β)
    H = zeros(T, p, p)
    for gc in qc_model.data
        d = gc.n # number of observations for current sample
        # GLM term
        H -= Transpose(gc.X) * Diagonal(gc.w2) * gc.X
        # 2nd term
        res = gc.res # d × 1 standardized residuals
        ∇resβ = gc.∇resβ # d × p
        Γ = zeros(T, d, d)
        for k in 1:gc.m # loop over variance components
            Γ .+= qc_model.θ[k] .* gc.V[k]
        end
        denom = 1 + 0.5 * (res' * Γ * res)
        H -= (∇resβ' * Γ * res) * (∇resβ' * Γ * res)' / denom^2
        # 3rd term
        H += (∇resβ' * Γ * ∇resβ) / denom
    end
    return H
end

# autodiff ∇²resβ (giving some kind of tensor)
∇²resβ_autodiff = x -> ForwardDiff.jacobian(∇resβ, x)

# this is d²rᵢₖ(β) needed for computing the 4th hessian term
function r_ik(β, k)
    res = resβ(β)
    return res[k]
end
r_ik(β) = r_ik(β, k)
∇²r_ik = x -> ForwardDiff.hessian(r_ik, x)

function full_hessian(qc_model::Union{GLMCopulaVCModel, NBCopulaVCModel})
    p = length(qc_model.β)
    T = eltype(qc_model.β)
    H = zeros(T, p, p)    
    # loop over samples
    for (i, gc) in enumerate(qc_model.data)
        d = gc.n # number of observations for current sample
        # GLM term
        H -= Transpose(gc.X) * Diagonal(gc.w2) * gc.X
        # 2nd term
        res = gc.res # d × 1 standardized residuals
        ∇resβ = gc.∇resβ # d × p
        Γ = zeros(T, d, d)
        for k in 1:gc.m # loop over variance components
            Γ .+= qc_model.θ[k] .* gc.V[k]
        end
        denom = 1 + 0.5 * (res' * Γ * res)
        H -= (∇resβ' * Γ * res) * (∇resβ' * Γ * res)' / denom^2
        # 3rd term
        H += (∇resβ' * Γ * ∇resβ) / denom
        # 4th term
        ek = zeros(d)    
        for k in 1:d
            # somehow need to define autodiff functions here, or else k is treated as fixed
            function resβ(β)
                η = X * β # d by 1
                μ = GLM.linkinv.(LogitLink(), η)
                varμ = GLM.glmvar.(Bernoulli(), μ)
                return (y - μ) ./ sqrt.(varμ) # d by 1
            end
            r_ik(β, k) = resβ(β)[k]
            r_ik(β) = r_ik(β, k)
            ∇²r_ik = x -> ForwardDiff.hessian(r_ik, x) 
            
            
            fill!(ek, 0)
            ek[k] = 1
            X = gc.X
            y = gc.y
#             @show ∇²r_ik(qc_model.β)
            H += (ek' * Γ * res * ∇²r_ik(qc_model.β)) / denom
        end
    end
    return H
end

full_hessian (generic function with 1 method)

In [29]:
# directly evaluating ∇²resβ (giving some kind of tensor)
Hββ = ∇²resβ_autodiff(qc_model.β)

15×3 Matrix{Float64}:
 -0.253406    0.525711    -0.572947
 -0.279741   -0.544617    -0.335183
 -0.263081    0.0212769    0.0200741
 -0.263562    0.0407485   -0.124028
 -0.272238   -0.253716    -0.280512
  0.525711   -1.09063      1.18862
 -0.544617   -1.06029     -0.652555
  0.0212769  -0.00172079  -0.00162351
  0.0407485  -0.00629998   0.0191756
 -0.253716   -0.236454    -0.261427
 -0.572947    1.18862     -1.29542
 -0.335183   -0.652555    -0.401613
  0.0200741  -0.00162351  -0.00153173
 -0.124028    0.0191756   -0.0583658
 -0.280512   -0.261427    -0.289037

In [30]:
# calculating hessian of r_ik for k = 1:p
k = 1
∇²r_ik(qc_model.β)

3×3 Matrix{Float64}:
 -0.253406   0.525711  -0.572947
  0.525711  -1.09063    1.18862
 -0.572947   1.18862   -1.29542

In [31]:
# calculating hessian of r_ik for k = 1:p
k = 2
∇²r_ik(qc_model.β)

3×3 Matrix{Float64}:
 -0.279741  -0.544617  -0.335183
 -0.544617  -1.06029   -0.652555
 -0.335183  -0.652555  -0.401613

In [33]:
# 2 term Hessian from math
two_terms_H = two_term_Hessian(qc_model)

3×3 Matrix{Float64}:
 -5800.35       28.5649     84.1043
    28.5649  -5155.84       59.5103
    84.1043     59.5103  -5164.75

In [34]:
# 3 term Hessian from math
three_term_hessian(qc_model)

3×3 Matrix{Float64}:
 -3684.56       28.4691     61.3107
    28.4691  -4268.18       48.266
    61.3107     48.266   -4284.97

In [35]:
# 3 term Hessian implemented by sarah
loglikelihood!(qc_model, true, true)
qc_model.Hβ

3×3 Matrix{Float64}:
 -3684.56       28.4691     61.3107
    28.4691  -4268.18       48.266
    61.3107     48.266   -4284.97

In [36]:
# 4 term Hessian from math
full_hessian(qc_model)

3×3 Matrix{Float64}:
 -2793.55       27.4235     44.8972
    27.4235  -3385.69       43.6875
    44.8972     43.6875  -3393.24

In [39]:
# autodiff Hessian
∇²logl = x -> ForwardDiff.hessian(
        autodiff_loglikelihood, x)
autodiff_H = ∇²logl(qc_model.β)

3×3 Matrix{Float64}:
 -2793.55       27.4235     44.8972
    27.4235  -3385.69       43.6875
    44.8972     43.6875  -3393.24

# Check $\nabla_{\theta}L$, $\nabla^2_{\theta}L$, and $\nabla_{\theta}\nabla_{\beta} L$ 

In [11]:
# Loglikelihood function friendly to autodiff
autodiff_loglikelihood(β) = QuasiCopula.loglikelihood(β, qc_model, z)

# autodiff Gradient
∇logl = x -> ForwardDiff.gradient(autodiff_loglikelihood, x)

# autodiff Hessian
∇²logl = x -> ForwardDiff.hessian(autodiff_loglikelihood, x)


#22 (generic function with 1 method)

First, check if `autodiff_loglikelihood` returns same answer as `QuasiCopula.loglikelihood!`

In [16]:
i = 1
z = convert(Vector{Float64}, @view(G[:, i]), center=true, scale=false)
fullβ = [qc_model.β; qc_model.θ; 0.0] # poisson or bernoulli
# fullβ = [qc_model.β; qc_model.θ; qc_model.τ; 0.0] # normal

@show autodiff_loglikelihood(fullβ)
@show QuasiCopula.loglikelihood!(qc_model, false, false);

autodiff_loglikelihood(fullβ) = -18033.163626812184
QuasiCopula.loglikelihood!(qc_model, false, false) = -18033.16362681217


### Check $\nabla_\theta L$

In [39]:
# autodiff (the 4th and 5th position stores the 2 gradient terms with respect to θ)
i = 5
z = convert(Vector{Float64}, @view(G[:, i]), center=true, scale=false)
fullβ = [qc_model.β; qc_model.θ; 0.0] # poisson or bernoulli
∇logl(fullβ)[4:5]

2-element Vector{Float64}:
 -1.1370986533992892e-6
  2.3139215938341664e-6

In [40]:
# mathematical formula
Ω = qc_model.data[i].V
m = length(Ω)
grad_math = zeros(m)
for i in 1:length(qc_model.data)
    r = qc_model.data[i].res
    Ω = qc_model.data[i].V
    b = [0.5r' * Ω[k] * r for k in 1:m]
    c = [0.5tr(Ω[k]) for k in 1:m]
    grad_math += b / (1 + qc_model.θ'*b) - c / (1 + qc_model.θ'*c)
end
grad_math

2-element Vector{Float64}:
 -1.137098653399271e-6
  2.3139215938342303e-6

### Check $\nabla_\theta^2 L$

In [37]:
# autodiff (the 4th and 5th position stores the 2 gradient terms with respect to θ)
i = 5
z = convert(Vector{Float64}, @view(G[:, i]), center=true, scale=false)
fullβ = [qc_model.β; qc_model.θ; 0.0] # poisson or bernoulli
∇²logl(fullβ)[4:5, 4:5]


2×2 Matrix{Float64}:
 1.25985e-15   1.0245e-15
 1.0245e-15   -9.38656e-15

In [62]:
# mathematical formula
Ω = qc_model.data[i].V
m = length(Ω)
hess_math = zeros(m, m)
for i in 1:length(qc_model.data)
    r = qc_model.data[i].res
    Ω = qc_model.data[i].V
    b = [0.5r' * Ω[k] * r for k in 1:m]
    c = [0.5tr(Ω[k]) for k in 1:m]
    hess_math += b*b' / (1 + qc_model.θ'*b)^2 - c*c' / (1 + qc_model.θ'*c)^2
end
hess_math

2×2 Matrix{Float64}:
 -1.25985e-15  -1.0245e-15
 -1.0245e-15    9.38656e-15

### Check $\nabla_\theta\nabla_\beta L$

In [44]:
# autodiff (the 4th and 5th position stores the 2 gradient terms with respect to θ)
i = 5
z = convert(Vector{Float64}, @view(G[:, i]), center=true, scale=false)
fullβ = [qc_model.β; qc_model.θ; 0.0] # poisson or bernoulli
∇²logl(fullβ)[1:3, 4:5]

3×2 Matrix{Float64}:
 -1.0073e-7   2.04979e-7
 -4.43245e-8  9.01974e-8
 -5.11492e-8  1.04085e-7

In [68]:
# mathematical formula
Ω = qc_model.data[i].V
m = length(Ω)
p = size(qc_model.data[i].X, 2)
hess_math = zeros(p, m)
for i in 1:length(qc_model.data)
    r = qc_model.data[i].res
    Ω = qc_model.data[i].V
    θ = qc_model.θ
    ∇resβ = qc_model.data[i].∇resβ
    b = [0.5r' * Ω[k] * r for k in 1:m]
    A = hcat([∇resβ' * Ω[k] * r for k in 1:m]...)
    hess_math += A ./ (1 + θ'*b) - (A*θ ./ (1 + θ'*b)^2) * b'
end
hess_math

3×2 Matrix{Float64}:
 -1.0073e-7   2.04979e-7
 -4.43245e-8  9.01974e-8
 -5.11492e-8  1.04085e-7

## Check $\frac{\partial^2\mu}{\partial \eta^2}$

In [19]:
"""
    mueta2(l::Link, η::Real)

Second derivative of the inverse link function `d^2μ/dη^2`, for link `L` at linear predictor value `η`.
I.e. derivative of the mueta function in GLM.jl
"""
function mueta2 end

mueta2(::IdentityLink, η::Real) = zero(η)
function mueta2(::LogitLink, η::Real)
    expabs = exp(-abs(η))
    denom = 1 + expabs
    return -expabs / denom^2 + 2expabs^2 / denom^3
end
mueta2(::LogLink, η::Real) = exp(η)

mueta2 (generic function with 3 methods)

In [22]:
# test mueta2 function
# l = IdentityLink()
# l = LogLink()
l = LogitLink()
η = 0.1234
μ = GLM.linkinv(l, η)

# mathematical hessian
@show mueta2(l, η)

# ForwardDiff Hessian
logit_mueta = η -> GLM.mueta(l, η)
mueta2_autodiff = x -> ForwardDiff.derivative(logit_mueta, x)
@show mueta2_autodiff(η);

mueta2(l, η) = -0.015346957645411913
mueta2_autodiff(η) = -0.015346957645411843


## Check $\frac{\partial\sigma^2}{\partial \mu}$

In [5]:
"""
    sigmaeta(D::Distribution, μ::Real)

Computes dσ²/dμ
"""
function sigmamu end

sigmamu(::Normal, μ::Real) = zero(μ)
sigmamu(::Bernoulli, μ::Real) = one(μ) - 2μ
sigmamu(::Poisson, μ::Real) = one(μ)

sigmamu (generic function with 3 methods)

## Check $\frac{\partial^2(\sigma^2)}{\partial \mu^2}$

In [6]:
"""
    sigmaμ2(D::Distribution, μ::Real)

Computes d²σ²/dμ²
"""
function sigmaμ2 end

sigmaμ2(::Normal, μ::Real) = zero(μ)
sigmaμ2(::Bernoulli, μ::Real) = -2
sigmaμ2(::Poisson, μ::Real) = zero(μ)

sigmaμ2 (generic function with 3 methods)

## Check $\nabla \mu$, $\nabla^2 \mu$, $\nabla \sigma^2$ and $\nabla^2 \sigma^2$

In [ ]:
"""
    ∇²μ(l::Link, X::Matrix, β::Vector)

Computes the Hessian of the mean function with respect to β
"""
function ∇²μ(l::Link, X::Matrix, β::Vector)
    η = X*β
    D = Diagonal(mueta2.(l, η))
    return X' * D * X
end

"""
    ∇²σ(d::Distribution, l::Link, X::Matrix, β::Vector)

Computes the Hessian of σ^2 function with respect to β
"""
function ∇²σ(d::Distribution, l::Link, X::Matrix, β::Vector)
    XXt = X * X'
    η = X*β
    μ = GLM.linkinv.(l, η)
    D = Diagonal(sigmamu2(d, μ)*GLM.mueta.(d, μ).^2 + sigmamu.(d, μ)*mueta2.(l, η))
    return X' * D * X
end

## Show that we can compute $∇resβ$ generally, although we don't do so in practice

In [27]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

# objective
function resβ(β)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogLink(), η)
    varμ = GLM.glmvar.(Poisson(), μ)
    return (y - μ) ./ sqrt.(varμ) # d by 1
end

# mathematical gradient
function ∇resβ(β::AbstractVector{T}) where T
    dist = Poisson()
    link = LogLink()
    
    d, p = size(X)
    ∇resβ = zeros(T, d, p)
    η = X * β # d by 1
    μ = GLM.linkinv.(link, η) # d by 1
    varμ = GLM.glmvar.(dist, μ) # d by 1
    res = (y - μ) ./ sqrt.(varμ) # d by 1
    dμ = GLM.mueta.(link, η) # d by 1
    for i in 1:p, j in 1:d
        varμ_j = varμ[j]
        x_ji = X[j, i]
        res_j = res[j]
        μ_j = μ[j]
        dμ_j = dμ[j]
        dμdβ = dμ_j * x_ji
        dσ2dβ = sigmamu(dist, μ_j) * dμdβ
        # in practice, we have update_∇resβ fucntions to compute ∇resβ[j, i]
        ∇resβ[j, i] = -inv(sqrt(varμ_j)) * dμdβ - 0.5 * res_j * inv(varμ_j) * dσ2dβ
    end
    return ∇resβ # d × p
end
∇²resβ_autodiff = x -> ForwardDiff.jacobian(∇resβ, x)

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)

# compare mathematical and numerical gradient
[vec(∇resβ(qc_model.β)) vec(∇resβ_autodiff(qc_model.β))]


15×2 Matrix{Float64}:
 -1.1211     -1.1211
 -1.14587    -1.14587
 -1.12998    -1.12998
 -1.13043    -1.13043
 -1.13862    -1.13862
  2.3258      2.3258
 -2.23085    -2.23085
  0.0913879   0.0913879
  0.174771    0.174771
 -1.06115    -1.06115
 -2.53478    -2.53478
 -1.37297    -1.37297
  0.0862215   0.0862215
 -0.53196    -0.53196
 -1.17322    -1.17322

## Compute $∇^2resβ$: Hessian of residual vector of sample $i$ at observation $k$

In [ ]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

function ∇²resβ_ik()
    dist = Poisson()
    link = LogLink()
    
    d, p = size(X)
    ∇²resβ_ik = zeros(T, p, p)
    η = X * β # d by 1
    μ = GLM.linkinv.(link, η) # d by 1
    varμ = GLM.glmvar.(dist, μ) # d by 1
    res = (y - μ) ./ sqrt.(varμ) # d by 1
    dμdη = GLM.mueta.(link, η) # d by 1
    for i in 1:p, j in 1:p
        varμ_j = varμ[j]
        invσ = inv(sqrt(varμ_j))
        x_ji = X[j, i]
        res_j = res[j]
        μ_j = μ[j]
        dμdη_j = dμdη[j]
        dμdβ = dμdη_j * x_ji
        dσ2dβ = sigmamu(dist, μ_j) * dμdβ
        # assemble 5 terms
        ∇²resβ_ik[j, i] = -invσ * ?? + 
                          0.5invσ^3 * dσ2dβ * dμdβ' - 
                          0.5 * res_j * inv(varμ_j) * ?? + 
                          0.5invσ^3 * dμdβ * dσ2dβ' + 
                          0.75res_j * inv(varμ_j^2) * dσ2dβ * dσ2dβ'
    end
    return ∇²resβ_ik # p × p
end

# autodiff hessian by computing autodiff gradient of ∇resβ
∇²resβ_autodiff = x -> ForwardDiff.jacobian(∇resβ, x)
